In [1]:
import pandas as pd

In [3]:
blog = pd.read_csv('blog_total.csv')
recommend = pd.read_csv('wine_recommend.csv')
month = pd.read_csv('wine_of_the_month_final.csv')

In [119]:
total = pd.DataFrame(columns=['name_ko', 'name_en', 'price', 'type', 'country', 'city', 'aroma', 'food', 'rate_expert', 'rate_consumer', 'alcohol', 'sugar', 'acidity', 'body', 'tannin', 'maker\'s_note', 'tasting_note', 'posting_title', 'posting_text', 'wine_title', 'wine_text', 'hashtag', 'blog_text'])

In [81]:
blog.columns

Index(['Name', 'Text'], dtype='object')

In [88]:
recommend.columns

Index(['한글 이름', '영문 이름', '포스팅 제목', '포스팅 본문', '와인 소개 제목', '와인 소개 본문', '해시태그'], dtype='object')

In [85]:
month.columns

Index(['한글 이름', '영문 이름', '가격', '종류', '나라', '도시', '아로마', '음식매칭', '전문가 평점',
       '소비자 평점', '알코올', '당도', '산도', '바디', '타닌', '메이커스 노트', '테이스팅 노트'],
      dtype='object')

* month의 중복 와인 제거

In [180]:
for name in month['영문 이름'].value_counts()[month['영문 이름'].value_counts() == 2].index:
    month.drop(month[month['영문 이름'] == name].index[-1], inplace=True)

In [194]:
month.index = range(len(month))
month['영문 이름'].value_counts()

Dr.Loosen, Riesling Sparkling                            1
Braida, Montebruna Barbera d'Asti                        1
Delicato, 1924 Bourbon Barrel Aged Cabernet Sauvignon    1
Coto de Imaz Gran Reserva                                1
Robert Mondavi, Cabernet Sauvignon                       1
                                                        ..
M. Chapoutier, Deschants Saint Joseph                    1
Casas del Toqui, Court Rollan Blanc de Blancs            1
Alta Vista, Premium Malbec                               1
Hardys, Limited Cellar Release Chardonnay                1
Chateau La Louviere White                                1
Name: 영문 이름, Length: 306, dtype: int64

* recommend의 중복 와인 제거

In [188]:
for name in recommend['영문 이름'].value_counts()[recommend['영문 이름'].value_counts() >= 2].index:
    recommend.drop(recommend[recommend['영문 이름'] == name].index[1:], inplace=True)

In [197]:
recommend.index = range(len(recommend))
recommend['영문 이름'].value_counts()

Frei Brothers, Pinot Noir                  1
Villa Jolanda, ‘I LOVE YOU’ Moscato        1
Billecart Salmon, Extra Brut               1
Lancers White & Rose                       1
Gerard Bertrand, Reserve Speciale Syrah    1
                                          ..
Lanson, Le Black Reserve                   1
Apaltagua, Envero Gran Reserva             1
Cattin, Sauvage Cremant d’Alsace Brut      1
Animus Vinho Verde                         1
Burmester Tawny Port                       1
Name: 영문 이름, Length: 581, dtype: int64

* total에 month 합치기

In [120]:
for i in range(len(month)):
    for column_total, column_month in zip(total.columns[:-5], month.columns):
        total.loc[i, column_total] = month.loc[i, column_month]

* total에 blog 합치기

In [234]:
total.blog_text = ''

for i in range(len(blog)):
    name, text = blog.loc[i]
    if name in total.name_en.values:
        total.loc[total[total.name_en == name].index[0], 'blog_text'] += str(text) + '\n\n\n'
    else:
        idx = len(total)
        total.loc[idx, 'name_en'] = name
        total.loc[idx, 'blog_text'] = str(text) + '\n\n\n'

* total에 recommand 합치기

In [244]:
for i in range(len(recommend)):
    name = recommend.loc[i, '영문 이름']
    if name in total.name_en.values:
        idx = total[total.name_en == name].index[0]
        total.loc[idx, 'posting_title'] = recommend.loc[i, '포스팅 제목']
        total.loc[idx, 'posting_text'] = recommend.loc[i, '포스팅 본문']
        total.loc[idx, 'wine_title'] = recommend.loc[i, '와인 소개 제목']
        total.loc[idx, 'wine_text'] = recommend.loc[i, '와인 소개 본문']
        total.loc[idx, 'hashtag'] = recommend.loc[i, '해시태그']
    else:
        idx = len(total)
        total.loc[idx, 'name_ko'] = recommend.loc[i, '한글 이름']
        total.loc[idx, 'name_en'] = recommend.loc[i, '영문 이름']
        total.loc[idx, 'posting_title'] = recommend.loc[i, '포스팅 제목']
        total.loc[idx, 'posting_text'] = recommend.loc[i, '포스팅 본문']
        total.loc[idx, 'wine_title'] = recommend.loc[i, '와인 소개 제목']
        total.loc[idx, 'wine_text'] = recommend.loc[i, '와인 소개 본문']
        total.loc[idx, 'hashtag'] = recommend.loc[i, '해시태그']

In [248]:
total.to_csv('total.csv')